#### DB Task

##### Subtask 1

In [1]:
import sqlite3
import pandas as pd

###### Printing each DF

In [2]:
with sqlite3.connect('database.db') as conn:
    clients_df = pd.read_sql('SELECT * FROM main.clients', conn)

clients_df

,client_id,name,age,city,account_number
0,1,Василий,34,Минск,1001
1,2,Екатерина,28,Москва,1002
2,3,Мария,45,Гродно,1003
3,4,Михаил,37,Витебск,1004
4,5,Евгения,29,Минск,1005


In [3]:
with sqlite3.connect('database.db') as conn:
    balance_df = pd.read_sql("SELECT * FROM main.balance", conn)

balance_df

,client_id,amount
0,1,1000
1,2,1500
2,3,800
3,4,2000
4,5,1200


In [4]:
with sqlite3.connect('database.db') as conn:
    transactions_df = pd.read_sql("SELECT * FROM main.transactions", conn)
    
transactions_df

,transaction_id,client_id,date,amount,type,description
0,1,1,2025-03-01,150,deposit,Salary
1,2,1,2025-03-02,-50,withdrawal,ATM withdrawal
2,3,2,2025-03-01,200,deposit,Freelance payment
3,4,2,2025-03-03,-30,withdrawal,Grocery store
4,5,3,2025-03-02,100,deposit,Transfer from friend
5,6,3,2025-03-04,-20,withdrawal,Coffee shop
6,7,4,2025-03-03,300,deposit,Bonus
7,8,4,2025-03-05,-70,withdrawal,Restaurant
8,9,5,2025-03-04,250,deposit,Investment return
9,10,5,2025-03-06,-40,withdrawal,Shopping


In [5]:
print(clients_df[clients_df['city'] == 'Минск']['age'].mean())

31.5


##### Subtask 2

In [6]:
high_balance_df = balance_df[balance_df['amount'] > 1000]
high_balance_df

,client_id,amount
1,2,1500
3,4,2000
4,5,1200


##### Subtask 3

In [7]:
types_df = transactions_df.groupby('type')['type'].count()
types_df

type
deposit       5
withdrawal    5
Name: type, dtype: int64

##### Subtask 4

In [8]:
transactions_df['amount'] = pd.to_numeric(transactions_df['amount'], errors='coerce')
print(transactions_df.groupby('client_id')['amount'].min().sort_values(ascending=True))

client_id
4   -70
1   -50
5   -40
2   -30
3   -20
Name: amount, dtype: int64


##### Subtask 5

In [9]:
number_of_transactions_df = (transactions_df.groupby('client_id').count()
                             .sort_values(ascending=False, by='client_id'))

number_of_transactions_df.iloc[:3]

,transaction_id,date,amount,type,description
client_id,,,,,
5,2,2,2,2,2
4,2,2,2,2,2
3,2,2,2,2,2


##### Subtask 6

In [10]:
merged_df = transactions_df.merge(balance_df, how='left', on='client_id')

merged_df

,transaction_id,client_id,date,amount_x,type,description,amount_y
0,1,1,2025-03-01,150,deposit,Salary,1000
1,2,1,2025-03-02,-50,withdrawal,ATM withdrawal,1000
2,3,2,2025-03-01,200,deposit,Freelance payment,1500
3,4,2,2025-03-03,-30,withdrawal,Grocery store,1500
4,5,3,2025-03-02,100,deposit,Transfer from friend,800
5,6,3,2025-03-04,-20,withdrawal,Coffee shop,800
6,7,4,2025-03-03,300,deposit,Bonus,2000
7,8,4,2025-03-05,-70,withdrawal,Restaurant,2000
8,9,5,2025-03-04,250,deposit,Investment return,1200
9,10,5,2025-03-06,-40,withdrawal,Shopping,1200


In [11]:
merged_df['amount_x'] = pd.to_numeric(merged_df['amount_x'], errors='coerce')
merged_df['amount_y'] = pd.to_numeric(merged_df['amount_y'], errors='coerce')

In [12]:
merged_df['percentage'] = merged_df['amount_x'] * 100 / merged_df['amount_y']

merged_df

,transaction_id,client_id,date,amount_x,type,description,amount_y,percentage
0,1,1,2025-03-01,150,deposit,Salary,1000,15.000000
1,2,1,2025-03-02,-50,withdrawal,ATM withdrawal,1000,-5.000000
2,3,2,2025-03-01,200,deposit,Freelance payment,1500,13.333333
3,4,2,2025-03-03,-30,withdrawal,Grocery store,1500,-2.000000
4,5,3,2025-03-02,100,deposit,Transfer from friend,800,12.500000
5,6,3,2025-03-04,-20,withdrawal,Coffee shop,800,-2.500000
6,7,4,2025-03-03,300,deposit,Bonus,2000,15.000000
7,8,4,2025-03-05,-70,withdrawal,Restaurant,2000,-3.500000
8,9,5,2025-03-04,250,deposit,Investment return,1200,20.833333
9,10,5,2025-03-06,-40,withdrawal,Shopping,1200,-3.333333


In [13]:
suspicious_df = merged_df[(merged_df['percentage'] > 15) |
                          (merged_df['percentage'] < -10)]

suspicious_df

,transaction_id,client_id,date,amount_x,type,description,amount_y,percentage
8,9,5,2025-03-04,250,deposit,Investment return,1200,20.833333
